# Corporacion Favorita - New Superb Forecasting Model - 

## Data Preperation Pipeline

Made by 4B Consultancy (Janne Heuvelmans, Georgi Duev, Alexander Engelage, Sebastiaan de Bruin) - 2024

In this data pipeline, the data used for forecasting item unit_sales will be processed and finalized before being imported in the machine learning model.   
The following steps are made within this notebook:  

>-0. Import Packages 
     
>-1. Load and optimize raw data  
    -1.1. Functions - Creation of downcast and normalize functions for initial data load  
    -1.2. Functions - Import raw data from local path  
    -1.3. Importing raw data  
       
>-2. Cleaning data (functions)  
    -2.1. Return list containing stores with less then 1670 operational days with sales  
    -2.2. Return list containing stores with cluster=10 in stores df  
    -2.3. Function to exclude stores with less then 1670 sales days and related to cluster 10 
 
>-3. Excluding data based on exploratory data analyses (functions)  
    -3.1. Function (partly optional) - Excluding stores based on sales units and on cluster type 10    
    -3.2. Function - Exclude holiday event related to the "Terromoto" volcano event  

>-4. Enriching datasets for further analysis (functions)  
    -4.1. Function - Determining holidays per store     
    -4.2. Function - Determining a count per type of holiday per store  
    -4.3. Function - Constructing a cartesian sales dataset for each store based on the maximum sales daterange  

>-5. Constructing final dataset

The structure of this notebook was inspired by:
https://hamilton.dagworks.io/en/latest/how-tos/use-in-jupyter-notebook/ 

## 0. Import packages

In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import sys
import altair as alt
import vegafusion as vf
import sklearn
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline, make_pipeline

## 1. Load and optimize raw data

### 1.1. Functions - Creation of downcast and normalize functions for initial data load
Update formatting of features to optimize memory and standardize column names.  
Furthermore, get basic information on loaded data and print back to user.  

1.1.1. Optimize memory by:  
- a) Remove spaces from column names.    
- b) Downcasting objects, integers and floats.  
- c) Standardize date columns to datetime format.

In [2]:
# Data memory optimization function 1 - Removing spaces from the column names
def standardize_column_names(s):
    """Removes spaces from the column names."""

    return s.replace(" ", "")

# Data memory optimization function 2 - Changing datatypes to smaller ones (downcasting)
def optimize_memory(df):
    """Optimize memory usage of a DataFrame by converting object columns to categorical
    and downcasting numeric columns to smaller types."""

    # Change: Objects to Categorical.
    object_cols = df.select_dtypes(include="object").columns
    if not object_cols.empty:
        print("Change: Objects to Categorical")
        df[object_cols] = df[object_cols].astype("category")

    # Change: Convert integers to smallest signed or unsigned integer and floats to smallest.
    for col in df.select_dtypes(include=["int"]).columns:
        if (df[col] >= 0).all():  # Check if all values are non-negative
            df[col] = pd.to_numeric(
                df[col], downcast="unsigned"
            )  # Downcast to unsigned
        else:
            df[col] = pd.to_numeric(df[col], downcast="integer")  # Downcast to signed

    # Downcast float columns
    for col in df.select_dtypes(include=["float"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    return df

# Data memory optimization function 4 - Transform date-related columns to datetime format.
def transform_date_to_datetime(df, i):
    """Transform date-related columns to datetime format."""
    if i != 0:
        if "date" in df.columns:
            print("Change: Transformed 'date' column to Datetime Dtype")
            df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None).dt.floor('D')

    return df

1.1.2. Return basic information on each dataframe:  
- a) Information on the number of observation and features.  
- b) Information on the optimized size of the dataframe. 

In [3]:
# Getting the basic information of the dataframe (number of observations and features, optimized size)
def df_basic_info(df, dataframe_name):
    print(
        f"The '{dataframe_name}' dataframe contains: {df.shape[0]:,}".replace(",", ".") + f" observations and {df.shape[1]} features."
    )
    print(
        f"After optimizing by downcasting and normalizing it has optimized size of    {round(sys.getsizeof(df)/1024/1024/1024, 2)} GB."
    )

### 1.2. Functions - Import raw data from local PATH
Create import data function and apply downcast, normalize functions and give basic information function within the importing function.

In [4]:
def f_get_data(i=0):

    # Define path.
    c_path = "C:/Users/sebas/OneDrive/Documenten/GitHub/Supermarketcasegroupproject/Group4B/data/raw/"

    # Identify file.
    v_file = (
        "history-per-year",  # 0
        "holidays_events",  # 1
        "items",  # 2
        "stores",  # 3
    )

    print(f"\nReading file {i}\n")

    # Load data.
    df = (
        pd.read_parquet(c_path + v_file[i] + ".parquet")
        .rename(columns=standardize_column_names)
        .pipe(optimize_memory)
        .pipe(transform_date_to_datetime, i)
    )

    # Return data.
    return df

### 1.3. Importing raw data
Importing parquet files with importing function (downcasting, normalizing and giving basic information)

In [5]:
# Sales History per year
df_sales = f_get_data(0)

# Holidays
df_holidays = f_get_data(1)

# Items
df_items = f_get_data(2)

# Stores
df_stores = f_get_data(3)


Reading file 0


Reading file 1

Change: Objects to Categorical
Change: Transformed 'date' column to Datetime Dtype

Reading file 2

Change: Objects to Categorical

Reading file 3

Change: Objects to Categorical


## 2. Cleaning data (functions)

### 2.1. Prepare and clean df_sales
Drop of columns "id", "year", "month", "day" and create a date column based on the columns "year" , "month" and "day".

In [6]:
# Prepare df_sales by cleaning up df for merging with holidays by dropping unneeded columns
def sales_cleaned(df_sales):
    df_sales["date"] = pd.to_datetime(df_sales[["year", "month", "day"]])
    df_sales = df_sales.drop(columns=["id", "year", "month", "day"])

    return df_sales

### 2.2. Prepare, clean and rename df_items
Renaming columns: "family" to "item_family" and  "class" to "item_class"

In [7]:
# Prepare df_items by cleaning up df by renaming columns for clearity in final df
def items_cleaned_renamed(df_items):

    df_items = df_items.rename(columns={"family": "item_family", "class": "item_class"})

    return df_items

### 2.3. Prepare, clean and rename df_stores
Drop of columns "state"  
Rename of columns "city" to "store_city", "cluster" to "store_cluster" and "type" to "store_type"

In [8]:
# Prepare df_stores by cleaning up df by dropping unneeded columns and rename columns for clearity in final df
def stores_cleaned_renamed(df_stores):

    df_stores = df_stores.drop(columns=["state"])

    df_stores = df_stores.rename(
        columns={"city": "store_city", "cluster": "store_cluster", "type": "store_type"}
    )

    return df_stores

## 3. Excluding data based on exploratory data analyses (functions)
Excluding sales data based on store sales availability  
Excluding holiday events related to the "Terromoto" volcano event

### 3.1. Function (partly optional) - Excluding stores based on sales units and on cluster type 10

3.1.1. Function (optional) - Return list containing stores with less then 1670 operational days with sales  
default parameter: store_exclusion_cutoff_number = 1670 days. Based on Exploratory data analysis, 17 stores do not have 1670 days of date present in the sales dataset and either are new stores are were closed for a significant number of days during the timeframe within the sales dataset. It might be functional to make the model only for stores that had sales for all dates (and not new) as that might influence model behavior. This function gives the flexibility as so the user can choose him/herself the cutoff point.

In [9]:
def stores_exclude_sales_days(df_sales, store_exclusion_cutoff_number=1670):

    # Group the sales data by store and date
    df_sales_grouped = (
        df_sales.groupby(["store_nbr", "date"]).agg({"unit_sales": "sum"}).reset_index()
    )

    # Count the number of daily sale records per store
    store_count = df_sales_grouped["store_nbr"].value_counts()

    # Get stores with counts less than the exclusion cutoff
    store_count_exclusion = store_count[store_count < store_exclusion_cutoff_number]

    # Get the list of store numbers to be excluded
    list_excluded_stores_sales_days = store_count_exclusion.index.tolist()
    
    return list_excluded_stores_sales_days

3.1.2. Function - Return list containing stores with cluster=10 in stores df  
From our exploratory data analysis we found that cluster 10 had data issues as it was the only cluster that could was assigned to multiple storetypes. Therefore and because these stores are not part of the top 10 in terms of unit sales, we excluded all stores assigned to cluster 10.

In [10]:
def stores_exclude_cluster(df_stores, cluster_number=10):

    # Get the list of store numbers that belong to cluster 10

    list_stores_cluster_10 = df_stores[df_stores["cluster"] == cluster_number][
        "store_nbr"
    ].tolist()

    return list_stores_cluster_10

3.1.3. Function - Exclude stores with less then X sales days and stores related to cluster 10  

In [11]:
def df_sales_cleaned_stores(df_sales, df_stores, store_exclusion_cutoff_number=1670):

    # Excluded less then 1670 salesdays
    list_excluded_stores_sales_days = stores_exclude_sales_days(
        df_sales, store_exclusion_cutoff_number
    )

    df_sales = df_sales.drop(
        df_sales[df_sales["store_nbr"].isin(list_excluded_stores_sales_days)].index
    )

    # Cluster 10
    list_stores_cluster_10 = stores_exclude_cluster(df_stores, cluster_number=10)

    df_sales = df_sales.drop(
        df_sales[df_sales["store_nbr"].isin(list_stores_cluster_10)].index
    )

    return df_sales

### 3.2. Function - Exclude holiday event related to the "Terromoto" volcano event

3.2.1. Function - Create dataframe based on df_holidays with only events containing "Terremoto Manabi"

In [12]:
def holiday_filter_vulcano_event(df_holidays, event_substring="Terremoto Manabi"):

    # Filter the DataFrame where 'description' contains the event_substring
    df_vulcano_event_filtered = df_holidays[
        df_holidays["description"].str.contains(event_substring)
    ]

    return df_vulcano_event_filtered

3.2.2. Function - Exclude the "Terremoto Manabi" from the df_holidays dataframe

In [13]:
def df_holidays_cleaned(df_holidays):

    # Exclude holiday_filter_vulcano_event function to return filtered df
    df_vulcano_event_filtered = holiday_filter_vulcano_event(df_holidays)

    # Filter the specific holiday events from the holiday DataFrame
    df_holidays = df_holidays.loc[
        ~df_holidays.index.isin(df_vulcano_event_filtered.index)
    ]

    return df_holidays

## 4. Enriching datasets for further analysis (functions)

### 4.1. Function - Determining holidays per store
The holidays dataset contains information on local, regional and national holidays. For each of these types, there is a different key/identifier that corresponds with the stores data found in df_stores (the raw data). To overcome this issue, three separate dataframes are made for each type of holiday where the data is merged (joined) with the stores dataframe. Thereafter, these dataframes are combined as to construct one big dataframe containing all the holidays per store.

4.1.1. Function - Make cleaned versions of the holidays and stores dataframe

In [14]:
# Prepare df_holiday and df_stores by cleaning up df for merging with holidays by dropping unneeded columns
def clean_holidays_stores_prep(df_holidays, df_stores):

    df_holidays_cleaned = df_holidays.drop(
        columns=[
            "description",
            "transferred",
        ]
    )

    df_stores_cleaned = df_stores.drop(columns=["cluster", "type"])

    return df_holidays_cleaned, df_stores_cleaned

4.1.2. Function - Create a dataframe with all the local holidays per store

In [15]:
def holidays_prep_local(df_holidays, df_stores):

    df_holidays_cleaned, df_stores_cleaned = clean_holidays_stores_prep(
        df_holidays, df_stores
    )

    # select locale 'Local' from holiday df and merge with city stores df
    df_holidays_local = df_holidays_cleaned[df_holidays_cleaned["locale"] == "Local"]

    df_holidays_prep_local = df_holidays_local.merge(
        df_stores_cleaned, left_on="locale_name", right_on="city", how="left"
    )

    return df_holidays_prep_local

4.1.3. Function - Create a dataframe with all the regional holidays per store

In [16]:
def holidays_prep_regional(df_holidays, df_stores):

    df_holidays_cleaned, df_stores_cleaned = clean_holidays_stores_prep(
        df_holidays, df_stores
    )

    # select locale 'Regional' from holiday df and merge with state stores df
    df_holidays_regional = df_holidays_cleaned[
        df_holidays_cleaned["locale"] == "Regional"
    ]

    df_holidays_prep_regional = df_holidays_regional.merge(
        df_stores_cleaned, left_on="locale_name", right_on="state", how="left"
    )

    return df_holidays_prep_regional

4.1.4. Function - Create a dataframe with all the national holidays per store

In [17]:
def holidays_prep_national(df_holidays, df_stores):

    df_holidays_cleaned, df_stores_cleaned = clean_holidays_stores_prep(
        df_holidays, df_stores
    )

    # Select locale 'Regional' from holiday df and merge with national stores df
    df_holidays_national = df_holidays_cleaned[
        df_holidays_cleaned["locale"] == "National"
    ]

    # Create extra column for merge on "Ecuador"
    df_stores_cleaned["national_merge"] = "Ecuador"

    df_holidays_prep_national = df_holidays_national.merge(
        df_stores_cleaned, left_on="locale_name", right_on="national_merge", how="left"
    )

    # Drop newly created column national_merge, not needed further
    df_holidays_prep_national = df_holidays_prep_national.drop(
        columns=["national_merge"]
    )

    return df_holidays_prep_national

4.1.5. Function - Create a dataframe that merges all the separate dataframe for each type of holiday and store combination

In [18]:
def holidays_prep_merged(df_holidays, df_stores):

    # Load prep functions from local, Regional and National df's
    df_holidays_prep_local = holidays_prep_local(df_holidays, df_stores)

    df_holidays_prep_regional = holidays_prep_regional(df_holidays, df_stores)

    df_holidays_prep_national = holidays_prep_national(df_holidays, df_stores)

    # Combine local, regional and national dataframes into 1 merged dataframe
    df_holidays_merged = pd.concat(
        [df_holidays_prep_local, df_holidays_prep_regional, df_holidays_prep_national]
    )

    # Clean df_holidays_merged by dropping locale_name", "city", "state"
    df_holidays_merged = df_holidays_merged.drop(
        columns=["locale_name", "city", "state"]
    )

    # Rename 'type' of holiday to 'holiday_type'
    df_holidays_merged = df_holidays_merged.rename(
        columns={"type": "holiday_type", "locale": "holiday_locale"}
    )

    return df_holidays_merged

### 4.2. Function - Determining a count per type of holiday per store
The dataframe resulting from the function described in 4.1. gives duplicate values because there sometimes are multiple holidays on one date. Duplicate values per date would result in multiple sales rows for each date, making it not workable. Therfore, we transform the holiday and stores combination to contain 3 columns (for each type of holiday, namely, local, regional and national) that count the amount of holidays found for a specific date. Thereby we create a unique list of date and store combinations for all the holidays within the dataset.

4.2.1. Function - Creating unique combination of store and date with three count columns for each type of holiday

In [61]:
def holidays_prep_merged_grouped(df_holidays, df_stores):

    # Merge the holiday dataframes and clean the merged dataframe
    df_holidays_merged = holidays_prep_merged(df_holidays, df_stores)

    # Group by date and store_nbr and count the number of holidays per date per store
    df_holidays_merged_grouped = df_holidays_merged.pivot_table(
        index=['date', 'store_nbr'],
        columns='holiday_locale',
        values='holiday_type', 
        aggfunc='count'
    ).reset_index()

    # The nature of the pivot function causes it to append date and store_nbrs for all possible combinations and thus not only the date and store combinations that we originally had in our data, we will conduct an inner join with the original data to get the original date and store_nbr combinations back

    # Remove the name of the columns
    df_holidays_merged_grouped.columns.name = None

    # Rename the columns to countoflocalholidays, countofregionalholidays, countofnationalholidays
    df_holidays_merged_grouped = df_holidays_merged_grouped.rename(columns={
        'Local': 'countoflocalholidays', 
        'Regional': 'countofregionalholidays', 
        'National': 'countofnationalholidays'
    })

    # Fill NaN values with 0
    df_holidays_merged_grouped = df_holidays_merged_grouped.fillna(0)

    # Let's do an inner join with the original data to get the original date and store_nbr combinations back. Therefore we need to make another dataframe.
    df_holidays_merged_grouped_inner = holidays_prep_merged(df_holidays, df_stores)
    df_holidays_merged_grouped_inner = df_holidays_merged_grouped_inner.groupby(['date', 'store_nbr']).size().reset_index().drop(columns=0)

    df_holidays_merged_grouped = df_holidays_merged_grouped.merge(df_holidays_merged_grouped_inner, on=['date', 'store_nbr'], how='inner')

    # Convert the count columns to integer
    df_holidays_merged_grouped = df_holidays_merged_grouped.astype({'countoflocalholidays': int, 'countofregionalholidays': int, 'countofnationalholidays': int})

    print(f'In the orignal unioned holiday dataframe, df_holidays_merged we found (including duplicates) {df_holidays_merged.shape[0]} rows')
    print(f'In our new adjusted dataframe we have {df_holidays_merged_grouped.shape[0]} rows')
    print(f'Thus, we have removed {df_holidays_merged.shape[0] - df_holidays_merged_grouped.shape[0]} rows')

    # Might want to filter out the holiday dates that will never be in de salesdate range. However, they will be left out anyway when joining with the sales data.
    return df_holidays_merged_grouped

In [62]:
fixdf = holidays_prep_merged_grouped(df_holidays, df_stores)
fixdf = fixdf[fixdf['store_nbr'] == 52]

fixdf.shape

In the orignal unioned holiday dataframe, df_holidays_merged we found (including duplicates) 9950 rows
In our new adjusted dataframe we have 9601 rows
Thus, we have removed 349 rows


C:\Users\sebas\AppData\Local\Temp\ipykernel_7372\3396661543.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_holidays_merged_grouped = df_holidays_merged.pivot_table(


(174, 5)

In [58]:
def holidays_prep_merged_grouped1(df_holidays, df_stores):

    # Merge the holiday dataframes and clean the merged dataframe
    df_holidays_merged = holidays_prep_merged(df_holidays, df_stores)

    # Group by date and store_nbr and count the number of holidays per date per store
    df_holidays_merged_grouped = df_holidays_merged.pivot_table(
        index=['date', 'store_nbr'],
        columns='holiday_locale',
        values='holiday_type', 
        aggfunc='count',
        observed=True
    ).reset_index()

    # The nature of the pivot function causes it to append date and store_nbrs for all possible combinations and thus not only the date and store combinations that we originally had in our data, we will conduct an inner join with the original data to get the original date and store_nbr combinations back

    # Remove the name of the columns
    df_holidays_merged_grouped.columns.name = None

    # Rename the columns to countoflocalholidays, countofregionalholidays, countofnationalholidays
    df_holidays_merged_grouped = df_holidays_merged_grouped.rename(columns={
        'Local': 'countoflocalholidays', 
        'Regional': 'countofregionalholidays', 
        'National': 'countofnationalholidays'
    })

    # Fill NaN values with 0
    df_holidays_merged_grouped = df_holidays_merged_grouped.fillna(0)

    # Let's do an inner join with the original data to get the original date and store_nbr combinations back. Therefore we need to make another dataframe.
    # df_holidays_merged_grouped_inner = holidays_prep_merged(df_holidays, df_stores)
    # df_holidays_merged_grouped_inner = df_holidays_merged_grouped_inner.groupby(['date', 'store_nbr']).size().reset_index().drop(columns=0)
    # df_holidays_merged_grouped = df_holidays_merged_grouped.merge(df_holidays_merged_grouped_inner, on=['date', 'store_nbr'], how='inner')

    # Convert the count columns to integer
    df_holidays_merged_grouped = df_holidays_merged_grouped.astype({'countoflocalholidays': int, 'countofregionalholidays': int, 'countofnationalholidays': int})

    print(f'In the orignal unioned holiday dataframe, df_holidays_merged we found (including duplicates) {df_holidays_merged.shape[0]} rows')
    print(f'In our new adjusted dataframe we have {df_holidays_merged_grouped.shape[0]} rows')
    print(f'Thus, we have removed {df_holidays_merged.shape[0] - df_holidays_merged_grouped.shape[0]} rows')

    # Might want to filter out the holiday dates that will never be in de salesdate range. However, they will be left out anyway when joining with the sales data.
    return df_holidays_merged_grouped

In [59]:
fixdf1 = holidays_prep_merged_grouped1(df_holidays, df_stores)

fixdf1 = fixdf1[fixdf1['store_nbr'] == 52]

fixdf1.shape

In the orignal unioned holiday dataframe, df_holidays_merged we found (including duplicates) 9950 rows
In our new adjusted dataframe we have 9601 rows
Thus, we have removed 349 rows


(174, 5)

4.2.2. Function - Filling in NA values for each count column whenever no holiday could be found for a specific holiday date and store combination

In [20]:
#  Fill newly created NaN columns, due to holiday join, with 'no' on thates where there are now holidays
def holidays_fill_zero_normal(df,df_holidays_merged_grouped):
    """
    Fills the NaN values with 0 for all columns with type "int32" in the combined dataframe.
    It will only fill the columns that are in the original dataframe and not in the holiday dataframe.
    """ 
    columns_to_fill = df_holidays_merged_grouped.columns.intersection(df.columns)
    int32_columns_to_fill = [col for col in columns_to_fill if df_holidays_merged_grouped[col].dtype == 'int32']
    df[int32_columns_to_fill] = df[int32_columns_to_fill].fillna(0).astype('int32')
    
    return df

### 4.3. Function - Constructing a cartesian sales dataset for each store based on the maximum sales daterange
The df_sales dataset contains unit sales data for each store but not all stores have data for each date. To overcome this and make sure each date is present for each store we construct a new dataframe based on the minimum- and maximum date found within the sales dataframe. The result is thus a sales dataframe with each date, store and item combination for the whole timerange.

In [21]:
def filling_dates_cartesian(df_sales):

    # Create new df to include all daily dates in the range, filling missing dates with NaNs
    df = df_sales.copy()

    # Print first and last date of df
    print(f'First date in df: {df["date"].min()}')
    print(f'Last date in df:  {df["date"].max()}')

    # Calculate memory size and shape size of start df
    df_mem_start = sys.getsizeof(df)
    df_shape_start = df.shape[0] / 1e6
    print(
        f"Start size of df_sales:     {round(df_mem_start/1024/1024/1024, 2)} GB and start observations:     {round(df_shape_start, 1)} million."
    )

    # Create a complete date range for the entire dataset, it's a datetimeindex object 
    all_dates = pd.date_range(start=df["date"].min(), end=df["date"].max(), freq="D")

    # Create a multi-index from all possible combinations of 'item_nbr' and 'date'
    all_combinations = pd.MultiIndex.from_product(
        [df["store_nbr"].unique(), df["item_nbr"].unique(), all_dates],
        names=["store_nbr", "item_nbr", "date"],
    )

    print(
        f'The multi-index (all_combinations of store, date and item for the minimum and maximum dates found result in {round(all_combinations.shape[0]/1e6,1)} million rows, this is the amount of rows we expect in the final dataframe.'
    )

    # -----------------------------------------------------------------------------------------------------
    # Check for duplicates in the combination of 'store_nbr', 'item_nbr', and 'date'
    # This method is based on boolean indexing, when there's a true value for the duplicated method, it will return those rows to the duplicate_rows variable
    duplicate_rows = df[
        df.duplicated(subset=["store_nbr", "item_nbr", "date"], keep=False)
    ]
    if not duplicate_rows.empty:
        print(
            "Warning: Duplicate entries found in the combination of 'store_nbr', 'item_nbr', and 'date'."
        )
        print(f"Total dublicate rows {duplicate_rows.shape[0]}")
        print("-" * 71)

    # -----------------------------------------------------------------------------------------------------

    # Reindex the original DataFrame to include all combinations of 'store_nbr', 'item_nbr', and 'date'
    df_reindexed = df.set_index(["store_nbr", "item_nbr", "date"]).reindex(
        all_combinations
    )

    # Reset the index to turn the multi-index back into regular columns
    df_sales_cartesian = df_reindexed.reset_index()

    # Calculate memory size and shape size of final end df
    df_mem_end = sys.getsizeof(df_sales_cartesian)
    df_mem_change_perc = ((df_mem_end - df_mem_start) / df_mem_start) * 100
    df_mem_change = df_mem_end - df_mem_start

    df_shape_end = df_sales_cartesian.shape[0] / 1e6
    df_shape_change_perc = ((df_shape_end - df_shape_start) / df_shape_start) * 100
    df_shape_change = df_shape_end - df_shape_start

    print(
        f"Final size of the dataframe is:     {round(df_mem_end/1024/1024/1024, 2)} GB and end observations:       {round(df_shape_end, 1)} million."
    )
    print(
        f"Change in size of the dataframe is: {round(df_mem_change_perc, 2)} % and observations:           {round(df_shape_change_perc, 2)}     %."
    )
    print(
        f"Increased size of the dataframe is: {round(df_mem_change/1024/1024/1024, 2)} GB and increased observations: {round(df_shape_change, 1)} million."
    )

    return df_sales_cartesian

In [28]:
df_cartesiantest = filling_dates_cartesian(df_sales)

First date in df: 2013-01-01 00:00:00
Last date in df:  2017-08-15 00:00:00
Start size of df_sales:     3.04 GB and start observations:     125.5 million.
The multi-index (all_combinations of store, date and item for the minimum and maximum dates found result in 367.9 million rows, this is the amount of rows we expect in the final dataframe.
Final size of the dataframe is:     12.68 GB and end observations:       367.9 million.
Change in size of the dataframe is: 317.17 % and observations:           193.15     %.
Increased size of the dataframe is: 9.64 GB and increased observations: 242.4 million.


In [63]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           uint32        
 1   store_nbr    uint8         
 2   item_nbr     uint32        
 3   unit_sales   float32       
 4   onpromotion  boolean       
 5   day          uint8         
 6   year         category      
 7   month        category      
 8   date         datetime64[ns]
dtypes: boolean(1), category(2), datetime64[ns](1), float32(1), uint32(2), uint8(2)
memory usage: 3.0 GB


## 5. Constructing final dataset
In this step all the datasets will be merged together.

In [22]:
# Merge datasets
def merge_datasets(df_sales, df_items, df_stores, df_holidays):
    
    # Basic information of loaded data
    print("Step 1 - Importing, downcasting and normalizing data and optimizing memory, the following data has been imported.")
    df_basic_info(df_sales, "df_sales")
    print("""""")
    df_basic_info(df_items, "df_items")
    print("""""")
    df_basic_info(df_stores, "df_stores")
    print("""""")
    df_basic_info(df_holidays, "df_holidays")
    print("-" * 100)

    # Sales prep   
    print("Step 2 - Cleaning sales data and making a cartesian product of the sales data and the minimum and maximum dates found in the data.")
    df_sales = sales_cleaned(df_sales)
    df_sales = df_sales_cleaned_stores(df_sales, df_stores)
    df_sales_cartesian = filling_dates_cartesian(df_sales)
    print("-" * 100)

    # Holidays prep
    print("Step 3 - Cleaning holiday data and counting the number of holidays per date per store for each type of holiday (national, regional, local).")
    df_holidays = df_holidays_cleaned(df_holidays)
    df_holidays_merged_grouped = holidays_prep_merged_grouped(df_holidays, df_stores)
    print("-" * 100)

    # Stores prep
    print("Step 4 - Cleaning stores data (read: dropping unnecessary columns and renaming columns for clarity).")    
    df_stores = stores_cleaned_renamed(df_stores)
    print("-" * 100)

    # Items prep
    print("Step 5 - Cleaning items data (read: dropping unnecessary columns and renaming columns for clarity).")  
    df_items = items_cleaned_renamed(df_items)
    print("-" * 100)

    # Holidays merge on sales
    print("Step 6 - Adding holiday data to our cartesian product of sales data (with store, item and date combinations) and cleaning up null values for count of holiday columns.")  
    df_merged = df_sales_cartesian.merge(df_holidays_merged_grouped, on=["date", "store_nbr"], how="left")
    df_merged = holidays_fill_zero_normal(df_merged,df_holidays_merged_grouped)
    print("-" * 100)

    # Stores merged with sales+holidays
    print("Step 7 - Adding holiday data to our cartesian product of sales data (with store, item and date combinations) and cleaning up null values for count of holiday columns.")      
    df_merged = df_merged.merge(df_stores, on="store_nbr", how="left")
    print("-" * 100)
  
    # Change the dtype for item_nbr from uint32 to int32, during testing we found that the merge was not working properly with uint32
    df_merged["item_nbr"] = df_merged["item_nbr"].astype(int)
    df_items["item_nbr"] = df_items["item_nbr"].astype(int)
 
    # Items merged with sales+holidays+stores
    print("Step 8 - Adding items data to our cartesian product of sales data (with store, item and date combinations) and cleaning up null values for count of holiday columns. Remember, in our last step we added a lot of store information as well")   
    df_final = df_merged.merge(df_items, on="item_nbr", how="left")
    print("-" * 100)

    # Print some referential integrity checks to make sure we have the same amount of rows
    print(f'The amount of rows in the sales dataframe was {df_sales.shape[0] /1_000_000:.2f} million.')
    print(f'After making a cartesian product with date, store and item we had a total of {df_sales_cartesian.shape[0]/1_000_000:.2f} million rows.')
    print(f'After mergin with the holidays, stores, and items we have {df_final.shape[0]/1_000_000:.2f} million rows')
    print(f'The difference between the incoming and outgoing data from this function is {(df_final.shape[0] - df_sales.shape[0])/1_000_000:.2f} rows')
    print(f'If we compare the outgoing dataframe called "df_final" with the cartesian product of sales data and dates we see that the difference is {df_sales_cartesian.shape[0] - df_final.shape[0]} rows')
    print(f'If the difference is 0, we have a perfect match and we can continue with the next steps.')

# f"Final size of the dataframe is:     {round(df_mem_end/1024/1024/1024, 2)} GB and end observations:       {round(df_shape_end, 1)} million."
    return df_final

In [23]:
df_final = merge_datasets(df_sales, df_items, df_stores, df_holidays)  # --> 2.44 GB

Step 1 - Importing, downcasting and normalizing data and optimizing memory, the following data has been imported.
The 'df_sales' dataframe contains: 125.497.040 observations and 8 features.
After optimizing by downcasting and normalizing it has optimized size of    2.1 GB.

The 'df_items' dataframe contains: 4.100 observations and 4 features.
After optimizing by downcasting and normalizing it has optimized size of    0.0 GB.

The 'df_stores' dataframe contains: 54 observations and 5 features.
After optimizing by downcasting and normalizing it has optimized size of    0.0 GB.

The 'df_holidays' dataframe contains: 350 observations and 6 features.
After optimizing by downcasting and normalizing it has optimized size of    0.0 GB.
----------------------------------------------------------------------------------------------------
Step 2 - Cleaning sales data and making a cartesian product of the sales data and the minimum and maximum dates found in the data.
First date in df: 2013-01-02 0

C:\Users\sebas\AppData\Local\Temp\ipykernel_7372\3396661543.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_holidays_merged_grouped = df_holidays_merged.pivot_table(


In the orignal unioned holiday dataframe, df_holidays_merged we found (including duplicates) 8276 rows
In our new adjusted dataframe we have 8091 rows
Thus, we have removed 185 rows
----------------------------------------------------------------------------------------------------
Step 4 - Cleaning stores data (read: dropping unnecessary columns and renaming columns for clarity).
----------------------------------------------------------------------------------------------------
Step 5 - Cleaning items data (read: dropping unnecessary columns and renaming columns for clarity).
----------------------------------------------------------------------------------------------------
Step 6 - Adding holiday data to our cartesian product of sales data (with store, item and date combinations) and cleaning up null values for count of holiday columns.
----------------------------------------------------------------------------------------------------
Step 7 - Adding holiday data to our cartesian 

In [32]:
df_sales.shape[0]
df_final.shape[0]

245053620

In [24]:
# Sebastiaan code -
# What about the "onpromotion" column, seems that it has a lot of NaN values. Are these quality issues or is just that there's no promotion. 
# This issue didn't arrive after merging, it was there from the beginning (in the df_sales dataframe).
# You would expect that if there's no promotion going on the value to be "False"
df_sales1 = sales_cleaned(df_sales)

df_sales1_unique = df_sales1['onpromotion'].unique()

# 4 Data Manipulation

4.X Brainstorm ideas for imputing missing values


4.1.1 Create a date for all missing values/dates and keep the value of sales as NA

Action: Create all daily dates in the date range. Date range starts from first available date in df to last available date in df. Then filling missing dates with NaNs for per unique item per unique store


Troubleshooting of filling_dates_NaN function

In [25]:
df_final.info()
# Count nulls per column
null_counts = df_final.isnull().sum()

# Print results
for column, count in null_counts.items():
    print(f"Column '{column}' has {count} null values.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245053620 entries, 0 to 245053619
Data columns (total 14 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   store_nbr                uint8         
 1   item_nbr                 int32         
 2   date                     datetime64[ns]
 3   unit_sales               float32       
 4   onpromotion              boolean       
 5   countoflocalholidays     int32         
 6   countofnationalholidays  int32         
 7   countofregionalholidays  int32         
 8   store_city               category      
 9   store_type               category      
 10  store_cluster            uint8         
 11  item_family              category      
 12  item_class               uint16        
 13  perishable               uint8         
dtypes: boolean(1), category(3), datetime64[ns](1), float32(1), int32(4), uint16(1), uint8(3)
memory usage: 8.7 GB
Column 'store_nbr' has 0 null values.
Column 'it

In [26]:
df_final_store_54_item_129296 = df_final[(df_final["store_nbr"] == 54) & (df_final["item_nbr"] == 129296)]

df_final_store_54_item_129296.tail(50)

,store_nbr,item_nbr,date,unit_sales,onpromotion,countoflocalholidays,countofnationalholidays,countofregionalholidays,store_city,store_type,store_cluster,item_family,item_class,perishable
238329188,54,129296,2017-06-27,NaN,<NA>,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329189,54,129296,2017-06-28,1.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329190,54,129296,2017-06-29,3.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329191,54,129296,2017-06-30,1.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329192,54,129296,2017-07-01,3.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329193,54,129296,2017-07-02,3.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329194,54,129296,2017-07-03,2.0,True,1,0,0,El Carmen,C,3,GROCERY I,1032,0
238329195,54,129296,2017-07-04,4.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329196,54,129296,2017-07-05,1.0,True,0,0,0,El Carmen,C,3,GROCERY I,1032,0
238329197,54,129296,2017-07-06,1.0,False,0,0,0,El Carmen,C,3,GROCERY I,1032,0


In [27]:
# This is very memory costly!!! --> will result in huge df
df_merged_full_nan = filling_dates_NaN(df_merged)


KeyboardInterrupt



4.1.2 Fill newly created dates for non-sales columns using forward fill and backward fill --> items, stores, holidays columns

In [ ]:
# Fill missing values for non-sales columns using forward fill and backward fill --> items, stores, holidays
non_sales_columns = [
    "store_city",
    "store_type",
    "store_cluster",
    "item_family",
    "item_class",
    "perishable",
]

# To-do: test more in individual item level how this works
df_merged_full_nan[non_sales_columns] = df_merged_full_nan.groupby(
    ["item_nbr", "store_nbr"]
)[non_sales_columns].transform(lambda group: group.ffill().bfill())

In [ ]:
df_final_store_54_item_129296 = df_merged_full_nan[(df_merged_full_nan["store_nbr"] == 54) & (df_merged_full_nan["item_nbr"] == 129296)]

df_final_store_54_item_129296.tail(50)

4.2: Detect negative values

•	Action: Delete unit_sales if values are lower than zero --> N/A

In [ ]:
def negative_sales_cleaned(df):

    # Check the number of negative values before replacement
    before_replacement = (df["unit_sales"] < 0).sum()
    print(f"Number of negative values before replacement: {before_replacement}")

    # Create a boolean mask for the negative sales rows to create a 'boolean flag-list' containing all negative rows, used to filter full df_sales df
    negative_sales_mask = df["unit_sales"] < 0

    # Use the mask to update the flagged 'unit_sales' column in the original DataFrame
    df.loc[negative_sales_mask, "unit_sales"] = df.loc[
        negative_sales_mask, "unit_sales"
    ].where(df.loc[negative_sales_mask, "unit_sales"] >= 0, np.nan)

    # Check the number of negative values after replacement
    after_replacement = (df["unit_sales"] < 0).sum()
    print(f"Number of negative values after replacement: {after_replacement}")

    return df

In [ ]:
# Full merged df_merged_full

# df_sales_nan = negative_sales_cleaned(df_merged_full)

In [ ]:
# Check function with df_sales
df_sales_nan = negative_sales_cleaned(df_sales)

df_sales_nan_check = df_sales_nan[df_sales_nan["unit_sales"].isna()].sort_values(
    by=["date", "store_nbr", "item_nbr"]
)

df_sales_nan_check.head(30)

4.3 Define new, old and closed stores

•	Condition: sales for all items a given store and date are NA

•	Action: Impute with 0


In [ ]:
# Sum/Agg all sales group  by store, date
# --> Sum_sales > 0 then store_opened
# else --> closed --> inputed with 0


# TO-do: discuss about closed_store --> inpute with 0 or N/A?

4.4 New product

•	Before the very first sale of an item, all observations are kept as NA

•	After the very first sale of an item, we go to step 3:  


In [ ]:
# Sum/Agg all sales group  by item, date
# --> Sum_sales > 0 then first_sales_day of product
# else <first_sales_day of product --> delete unit_sales --> N/A

4.8  Stockout on store level

•      Perishable good: when there are missing values for two consecutive days for a given item per individual store 

•      Nonperishable goods: when there are missing values for 7 consecutive days for a given item and per individual store

•      Action: Impute with algorithm 


In [ ]:
#perishable good
if ['perishable'] == 1 and item_missing_count > 2 #-->  inpute with 0?

if ['perishable'] == 1 and item_missing_count <= 2 #-->  inpute with mean? or intrepolate?




    
#non-perishable good  
if ['perishable'] == 0 and and item_missing_count > 7 #-->  inpute with 0?

if ['perishable'] == 1 and item_missing_count <= 7 #-->  inpute with mean? or intrepolate?

In [ ]:
# Interpolate between missing datapoints --> sales

fillna(method="mean")

df["column_name"].interpolate(method="linear", inplace=True)

df["column_name"].interpolate(method="time", inplace=True)

df["column_name"].interpolate(method="polynomial", order=2, inplace=True)

# Interpolate missing values for the 'unit_sales' column
df["unit_sales"] = df.groupby(["store_nbr", "item_nbr"])["unit_sales"].apply(
    lambda group: group.interpolate(method="linear")
)

4.4 Missing sales data: Zero sales

•	All other cases

•	Action: Impute with 0

4.X Negative values imputing to 0

4.5 Promotional Data 

•   All missing values are interpreted a day with no promotion

•   Action: Inpute onpromotion N/A with False

In [ ]:
# Fill missing N/A values in boolean columns with False
def sales_fill_onpromotion(df):

    df["onpromotion"] = df["onpromotion"].fillna(False)

    return df


# To-do: when perform this function? Before filling_dates_NaN?

In [ ]:
# df_merged_test = sales_fill_onpromotion(df_merged)

# df_merged_test.head(10)

# 5 Feature construction

5.X Extracting datetime features

In [ ]:
def extract_datetime_features(df):
    """
    Extracting datetime features
    year, month, day of month, weekday (1-7), week number-year, week_year_date
    """
    df = df.copy()

    # Ensure the date column is sorted
    df = df.sort_values("date")

    # df["year"] = df["date"].dt.year
    # df["month"] = df["date"].dt.month
    # df["day"] = df["date"].dt.day

    # Adjusting weekday to start from 1 (Monday) to 7 (Sunday)
    df["weekday"] = df["date"].dt.dayofweek + 1

    # Adding week number-year feature
    df["week_number"] = df["date"].dt.isocalendar().week
    df["week_year"] = df["week_number"].astype(str).str.zfill(2) + df["year"].astype(
        str
    )

    # Convert week_year to datetime with monday as startdate of week
    df["week_year_date"] = pd.to_datetime(
        df["year"].astype(str) + df["week_number"].astype(str).str.zfill(2) + "1",
        format="%Y%W%w",
    )

    # Adding trend feature: number of weeks since the start of the dataset
    start_date = df["date"].min()
    df["weeks_since_start"] = ((df["date"] - start_date).dt.days / 7).astype(int)

    return df

In [ ]:
def extract_datetime_features(df):
    """
    Extracting datetime features:
    year, month, day of month, weekday (1-7), week number-year, and trend (weeks since start, starting at 1)
    """
    # Ensure the date column is sorted
    df = df.copy().sort_values("date")

    # Use isocalendar for consistent week-based calculations

    iso_calendar = df["date"].dt.isocalendar()

    # Year, Month, Day
    # df["year"] = iso_calendar.year
    # df["month"] = df["date"].dt.month
    # df["day"] = df["date"].dt.day

    # Weekday (1 = Monday, 7 = Sunday)
    df["weekday"] = iso_calendar.day

    # Week number
    df["week_number"] = iso_calendar.week

    # Week-year
    df["week_year"] = df["week_number"].astype(str).str.zfill(2) + df["year"].astype(
        str
    )

    # Convert week_year to datetime with monday as startdate of week
    df["week_year_date"] = pd.to_datetime(
        df["year"].astype(str) + df["week_number"].astype(str).str.zfill(2) + "1",
        format="%Y%W%w",
    )

    # First day of the ISO year containing the start date

    start_date = df["date"].min()
    start_year_first_day = datetime(start_date.year, 1, 1)

    # 'search' for first monday of year
    while start_year_first_day.isocalendar()[1] != 1:

        start_year_first_day = start_year_first_day + pd.Timedelta(days=1)

    ##Itemweek number
    # Weeks since start (aligned with ISO week numbers)
    df["weeks_since_start"] = (
        iso_calendar.week + (iso_calendar.year - start_year_first_day.year) * 52
    )

    # Adjust weeks_since_start to start from 1
    df["weeks_since_start"] = (
        df["weeks_since_start"] - df["weeks_since_start"].min() + 1
    )

    return df

In [ ]:
# df = extract_datetime_features(df)

5.X Promotion

The number of days a item was on promotion 

In [ ]:
# COPY FROM OLD NOTEBOOK
# TO-DO 1: transform with new df names
# TO-DO 2: total promotion days month --> week


def onpromotion_month_count(df):

    if "onpromotion" in df.columns:

        df["onpromotion_month_count"] = df.groupby(
            ["item_nbr", "store_nbr", "day", "month", "year"]
        )["onpromotion"].transform("sum")

        print(
            "Change: 'onpromotion' column transformed to 'onpromotion_month_count' feature."
        )
    else:

        print("The DataFrame does not contain an 'onpromotion' column.")

    return df

In [ ]:
df_0_agg = (
    onpromotion_month_count(df_0)  # Transformation to 'onpromotion_month_count' feature
    .drop(
        columns=["id", "date", "onpromotion"]
    )  # Drop unnecessary columns "id", "date", "onpromotion"
    .groupby(["month", "year", "store_nbr", "item_nbr"])
    .agg({"unit_sales": "sum", "onpromotion_month_count": "sum"})
    .reset_index()
)

5.X Store closed on 25-12 and 01-01 

STore closed in between when inputed with 0

--> can we also use this feature to include the excluded stores with >9 days data, due to closing or later openings?